In [34]:
import numpy as np

D = 784
L = 12
M = 10
a = np.random.uniform(-1, 1, D)
b = np.random.uniform(-1, 1, L)
x = np.random.randint(0, 2, (M, D))
w = np.random.uniform(-1, 1, (D, L))


# TODO: implement sum of G(z) over z in Num and Den
class LOGlike(object):
    """
    FORMAT:
    D = 784   # visible
    L = 12    # hidden
    a = np.random.uniform(0, 1, D)
    b = np.random.uniform(0, 1, L)
    x = np.random.randint(0, 2, (M, D))
    z = np.random.randint(0, 2, L)
    w = np.random.uniform(0, 1, (D, L))
    """
    def make_z_states(self):
        def get_bin(i, L):
            b = bin(i)[2:]
            return ("0" * (L - len(b))) + b
            #
        #####
        #
        self.zs = np.array([[int(bit) for bit in get_bin(i, self.L)] for i in range(2 ** self.L)])
        #
    #####
    #   
    def __init__(self, a, b, x, w, M):
        self.a  = a   # visible
        self.b  = b   # hidden
        self.D  = a.shape[0]
        self.L  = b.shape[0]
        self.xs = x
        self.w  = w
        self.M  = M
        self.q  = None
        self.H    = lambda i, z: self.a[i] + np.dot(self.w[i], self.zs[z])
        self.G    = lambda z:    np.exp(np.dot(self.b, self.zs[z]))
        self.esit = None
        self.zs   = None
        self.make_z_states()
        #
    #####
    #
    def run(self):
        """
        l_θ(x) = ln{sum_z[G(z) * exp(H_i(z) * x_i)]} - ln{Z}
        """
        
        def get_numerator(x):
            """
            ln{sum_z[G(z) * exp[dot(H_i(z) * x_i)]]}
            """
            
            numerator = np.sum([self.G(z) * np.prod([np.exp(self.xs[x][i] * self.H(i, z))
                                                                 for i in range(self.D)])
                                                                 for z in range(self.L)])
            return np.log(numerator)
            #
        #####
        #
        def get_denominator():
            self.q = np.array([np.mean([self.H(i, z) for i in range(self.D)]) for z in range(self.L)])
            self.q = np.mean(1 + np.exp(self.q))
            """
            ln(Z) = D * ln(q) + ln{sum_z[G(z) * prod_i[(1 + H_i(z)) / q]]}
            """
            
            esit   = np.sum([self.G(z) * np.prod([(1 + np.exp(self.H(i, z))) / self.q
                                                             for i in range(self.D)])
                                                             for z in range(self.L)])
            return self.D * np.log(self.q) + np.log(esit)
            #
        #####
        #
        lθ        = lambda x: get_numerator(x) - get_denominator()
        self.esit = np.mean([lθ(x) for x in range(self.M)])
        #
    #####
    #
    def print(self):
        return self.esit
        #
    #####
    #
#####

log_likehood0 = LOGlike(a, b, x, w, M)
log_likehood0.run()
log_likehood0.print()

-638.1601072767716